# Assignment Section 1: Generate postal code from Wiki

In [1]:
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize 
import pandas as pd

## Request the html and Use BeautifulSoup to extract the content

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
soup = BeautifulSoup(results.content)

## Extract the table and its headings

In [3]:
post_code_list = soup.find("table", attrs={"class": "wikitable sortable"})
post_code_list_data = post_code_list.tbody.find_all("tr")  

# Get all the headings of the table
headings = []
for th in post_code_list_data[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(th.text.replace('\n', ' ').strip())

print(headings)

['Postal Code', 'Borough', 'Neighbourhood']


## Generate the table on the web

In [4]:
data = []

for row in post_code_list_data[1:]:
    t_row = {}
    for td, th in zip(row.find_all('td'), headings):
        t_row[th] = td.text.replace('\n', '').strip()
    data.append(t_row)
post_table = pd.DataFrame(data)
post_table

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Drop the lines with Borough not assigned

In [5]:
new_table = post_table.loc[post_table['Borough']!='Not assigned',:]
new_table

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
# It looks there is no repeated Postal Code line in the table
len(pd.unique(new_table['Postal Code']))

103

## Answer of the Section 1

* Assign the not-assigned neighborhood with the borough, clean table and generate the final table and it's shape 

In [7]:
for neighbor, i in enumerate(new_table['Neighbourhood']):
    if neighbor == 'Not assigned':
        new_table.loc[i,'Neighbourhood'] = new_table.loc[i,'Borough']
new_table.reset_index(drop=True, inplace=True)
print('The number of rows of my dataframe is',new_table.shape[0])
new_table

The number of rows of my dataframe is 103


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Assignment Section 2: Merge Latitude and Longitude data in the postal code table

In [8]:
!pip install geocoder

     |████████████████████████████████| 98 kB 5.1 MB/s eta 0:00:011


In [9]:
import geocoder

In [10]:
lat = []
lng = []
for i in range(new_table.shape[0]):
    postal_code = new_table.loc[i,'Postal Code']
    Borough = new_table.loc[i,'Borough']
    address = '{}, Toronto, ON {}'.format(Borough, postal_code)
    g = geocoder.arcgis(address)
    lat.append(g.lat)
    lng.append(g.lng)

In [11]:
new_table['Latitude'] = lat
new_table['Longitude'] = lng

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


## Answer of the Section 2

In [12]:
new_table

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65820,-79.36842
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


# Assignment Section 3: Exploring and Clustering

## Import lib

In [13]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

     |████████████████████████████████| 94 kB 4.9 MB/s  eta 0:00:01


* Expand the table to separate neighborhoods

In [14]:
new_table

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65820,-79.36842
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


In [15]:
lat = []
lng = []
for i in range(new_table.shape[0]):
    postal_code = new_table.loc[i,'Postal Code']
    Borough = new_table.loc[i,'Borough']
    address = '{}, Toronto, ON {}'.format(Borough, postal_code)
    g = geocoder.arcgis(address)
    lat.append(g.lat)
    lng.append(g.lng)

In [16]:
toronto_table = pd.DataFrame(columns=new_table.columns)

for i in new_table.index:
    if (len(new_table.loc[i,'Neighbourhood'].split(','))==1):
        toronto_table = toronto_table.append(new_table.loc[[i]], ignore_index=True)
    elif (len(new_table.loc[i,'Neighbourhood'].split(','))>1):
        for neighbourhood in new_table.loc[i,'Neighbourhood'].split(','):
            neighbourhood = neighbourhood.strip()
            toronto_table = toronto_table.append(new_table.loc[[i]], ignore_index=True)
            toronto_table.iloc[-1,2] = neighbourhood
            
            postal_code = toronto_table.iloc[-1, 0]
            borough = toronto_table.iloc[-1, 1]
            neighbourhood = toronto_table.iloc[-1, 2]
            address = "{}, {}, Toronto, ON {}".format(neighbourhood, borough, postal_code)
            g = geocoder.arcgis(address)
            toronto_table.iloc[-1,3] = g.lat
            toronto_table.iloc[-1,4] = g.lng
    else:
        print("Error occurs at", i, ' row.')

In [17]:
toronto_table

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752450,-79.329910
1,M4A,North York,Victoria Village,43.730570,-79.313060
2,M5A,Downtown Toronto,Regent Park,43.660690,-79.360310
3,M5A,Downtown Toronto,Harbourfront,43.655120,-79.362640
4,M6A,North York,Lawrence Manor,43.714925,-79.449739
...,...,...,...,...,...
212,M8Z,Etobicoke,Mimico NW,43.617290,-79.498850
213,M8Z,Etobicoke,The Queensway West,43.620494,-79.528704
214,M8Z,Etobicoke,South of Bloor,43.649266,-79.502830
215,M8Z,Etobicoke,Kingsway Park South West,43.657385,-79.503383


In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_table['Borough'].unique()),
        toronto_table.shape[0]
    )
)

The dataframe has 10 boroughs and 217 neighborhoods.


In [19]:
address = ' Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_table['Latitude'], toronto_table['Longitude'], toronto_table['Borough'], toronto_table['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Request and Process info from Foursquare

### Define Foursquare credential and version

In [21]:
CLIENT_ID = 'OQWZRAUBBN5XX5CTG4QJNUTGB1PLERCI32GLZWVRG4JJTOMN' # your Foursquare ID
CLIENT_SECRET = '5J0EYVSGK2QSQXGA4TN5YPIIUY1NU0HDIRQNHRWWB4BXUUAR' # your Foursquare Secret
ACCESS_TOKEN = 'VH53ZCZMIQSN033ZU2VDUZD1CRA5GJQE1SKGVRWNNF1FJINZ' # your FourSquare Access Token
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OQWZRAUBBN5XX5CTG4QJNUTGB1PLERCI32GLZWVRG4JJTOMN
CLIENT_SECRET:5J0EYVSGK2QSQXGA4TN5YPIIUY1NU0HDIRQNHRWWB4BXUUAR


* Define the func to request venues info from Foursquare

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        print(len(venues_list))

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues, venues_list)

In [23]:
names = toronto_table['Neighbourhood'].tolist()
latitudes = toronto_table['Latitude'].tolist()
longitudes = toronto_table['Longitude'].tolist()
toronto_venues, venues_list = getNearbyVenues(names, latitudes=latitudes, longitudes=longitudes)

Parkwoods
1
Victoria Village
2
Regent Park
3
Harbourfront
4
Lawrence Manor
5
Lawrence Heights
6
Queen's Park
7
Ontario Provincial Government
8
Islington Avenue
9
Humber Valley Village
10
Malvern
11
Rouge
12
Don Mills
13
Parkview Hill
14
Woodbine Gardens
15
Garden District
16
Ryerson
17
Glencairn
18
West Deane Park
19
Princess Gardens
20
Martin Grove
21
Islington
22
Cloverdale
23
Rouge Hill
24
Port Union
25
Highland Creek
26
Don Mills
27
Woodbine Heights
28
St. James Town
29
Humewood-Cedarvale
30
Eringate
31
Bloordale Gardens
32
Old Burnhamthorpe
33
Markland Wood
34
Guildwood
35
Morningside
36
West Hill
37
The Beaches
38
Berczy Park
39
Caledonia-Fairbanks
40
Woburn
41
Leaside
42
Central Bay Street
43
Christie
44
Cedarbrae
45
Hillcrest Village
46
Bathurst Manor
47
Wilson Heights
48
Downsview North
49
Thorncliffe Park
50
Richmond
51
Adelaide
52
King
53
Dufferin
54
Dovercourt Village
55
Scarborough Village
56
Fairview
57
Henry Farm
58
Oriole
59
Northwood Park
60
York University
61
East Tor

* Get idea of the table containing the info

In [51]:
print(toronto_venues.shape)
toronto_venues.head()

(4978, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
3,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store
4,Victoria Village,43.73057,-79.31306,Vinnia Meats,43.730465,-79.307520,German Restaurant


In [52]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,14,14,14,14,14,14
Agincourt North,2,2,2,2,2,2
Albion Gardens,4,4,4,4,4,4
Alderwood,4,4,4,4,4,4
...,...,...,...,...,...,...
Woodbine Heights,17,17,17,17,17,17
York Mills,4,4,4,4,4,4
York Mills West,4,4,4,4,4,4


# Analysis

In [112]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="Category", prefix_sep=".")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Category.Accessories Store,Category.African Restaurant,Category.Airport,Category.Airport Food Court,Category.Airport Lounge,Category.Airport Service,Category.Airport Terminal,Category.American Restaurant,Category.Animal Shelter,Category.Antique Shop,Category.Aquarium,Category.Argentinian Restaurant,Category.Art Gallery,Category.Art Museum,Category.Arts & Crafts Store,Category.Asian Restaurant,Category.Athletics & Sports,Category.Auto Dealership,Category.Auto Garage,Category.Automotive Shop,Category.BBQ Joint,Category.Baby Store,Category.Badminton Court,Category.Bagel Shop,Category.Bakery,Category.Bank,Category.Bar,Category.Baseball Field,Category.Basketball Stadium,Category.Beach,Category.Bed & Breakfast,Category.Beer Bar,Category.Beer Store,Category.Belgian Restaurant,Category.Bike Shop,Category.Bike Trail,Category.Bistro,Category.Boat or Ferry,Category.Bookstore,Category.Boutique,Category.Brazilian Restaurant,Category.Breakfast Spot,Category.Brewery,Category.Bridal Shop,Category.Bridge,Category.Bubble Tea Shop,Category.Buffet,Category.Building,Category.Burger Joint,Category.Burrito Place,Category.Bus Line,Category.Bus Station,Category.Bus Stop,Category.Business Service,Category.Butcher,Category.Cafeteria,Category.Café,Category.Cajun / Creole Restaurant,Category.Camera Store,Category.Candy Store,Category.Cantonese Restaurant,Category.Caribbean Restaurant,Category.Carpet Store,Category.Cheese Shop,Category.Chinese Restaurant,Category.Chiropractor,Category.Chocolate Shop,Category.Church,Category.Clothing Store,Category.Cocktail Bar,Category.Coffee Shop,Category.College Arts Building,Category.College Auditorium,Category.College Cafeteria,Category.College Gym,Category.College Rec Center,Category.College Stadium,Category.College Theater,Category.Colombian Restaurant,Category.Comedy Club,Category.Comfort Food Restaurant,Category.Comic Shop,Category.Concert Hall,Category.Construction & Landscaping,Category.Convenience Store,Category.Cosmetics Shop,Category.Costume Shop,Category.Creperie,Category.Cuban Restaurant,Category.Cupcake Shop,Category.Curling Ice,Category.Dance Studio,Category.Daycare,Category.Deli / Bodega,Category.Dentist's Office,Category.Department Store,Category.Dessert Shop,Category.Dim Sum Restaurant,Category.Diner,Category.Discount Store,Category.Distribution Center,Category.Dive Bar,Category.Doctor's Office,Category.Dog Run,Category.Doner Restaurant,Category.Donut Shop,Category.Dumpling Restaurant,Category.Eastern European Restaurant,Category.Electronics Store,Category.Elementary School,Category.Escape Room,Category.Ethiopian Restaurant,Category.Event Space,Category.Falafel Restaurant,Category.Farm,Category.Farmers Market,Category.Fast Food Restaurant,Category.Field,Category.Filipino Restaurant,Category.Fish & Chips Shop,Category.Fish Market,Category.Flea Market,Category.Flower Shop,Category.Food & Drink Shop,Category.Food Court,Category.Food Service,Category.Food Truck,Category.Fountain,Category.French Restaurant,Category.Fried Chicken Joint,Category.Frozen Yogurt Shop,Category.Fruit & Vegetable Store,Category.Furniture / Home Store,Category.Gaming Cafe,Category.Garden,Category.Gas Station,Category.Gastropub,Category.Gay Bar,Category.General Entertainment,Category.General Travel,Category.German Restaurant,Category.Gift Shop,Category.Gluten-free Restaurant,Category.Golf Course,Category.Golf Driving Range,Category.Gourmet Shop,Category.Greek Restaurant,Category.Grocery Store,Category.Gym,Category.Gym / Fitness Center,Category.Gym Pool,Category.Harbor / Marina,Category.Hardware Store,Category.Health & Beauty Service,Category.Health Food Store,Category.Historic Site,Category.History Museum,Category.Hobby Shop,Category.Hockey Arena,Category.Home Service,Category.Hong Kong Restaurant,Category.Hookah Bar,Category.Hospital,Category.Hostel,Category.Hot Dog Joint,Category.Hotel,Category.Hotel Bar,Category.Hotpot Restaurant,Category.IT Services,Category.Ice Cream Shop,Category.Indian Restaurant,Category.Indie Mo

In [118]:
toronto_venues.loc[(toronto_venues['Venue Category']=='Neighborhood')]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
749,Central Bay Street,43.654860,-79.382340,Downtown Toronto,43.653232,-79.385296,Neighborhood
923,Richmond,43.649700,-79.382580,Downtown Toronto,43.653232,-79.385296,Neighborhood
1023,Adelaide,43.649700,-79.382580,Downtown Toronto,43.653232,-79.385296,Neighborhood
1406,Harbourfront East,43.642850,-79.380760,Harbourfront,43.639526,-79.380688,Neighborhood
2761,Lawrence Park,43.725540,-79.402320,Bedford Park,43.725188,-79.408463,Neighborhood
3262,Summerhill East,43.684931,-79.386953,Summerhill,43.682976,-79.389123,Neighborhood
3418,Grange Park,43.652250,-79.391360,Downtown Toronto,43.653232,-79.385296,Neighborhood


In [122]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index() # The mean is the frequency rate of the venue in all the venues found in this neighbourhood.
toronto_grouped

,Neighborhood,Category.Accessories Store,Category.African Restaurant,Category.Airport,Category.Airport Food Court,Category.Airport Lounge,Category.Airport Service,Category.Airport Terminal,Category.American Restaurant,Category.Animal Shelter,Category.Antique Shop,Category.Aquarium,Category.Argentinian Restaurant,Category.Art Gallery,Category.Art Museum,Category.Arts & Crafts Store,Category.Asian Restaurant,Category.Athletics & Sports,Category.Auto Dealership,Category.Auto Garage,Category.Automotive Shop,Category.BBQ Joint,Category.Baby Store,Category.Badminton Court,Category.Bagel Shop,Category.Bakery,Category.Bank,Category.Bar,Category.Baseball Field,Category.Basketball Stadium,Category.Beach,Category.Bed & Breakfast,Category.Beer Bar,Category.Beer Store,Category.Belgian Restaurant,Category.Bike Shop,Category.Bike Trail,Category.Bistro,Category.Boat or Ferry,Category.Bookstore,Category.Boutique,Category.Brazilian Restaurant,Category.Breakfast Spot,Category.Brewery,Category.Bridal Shop,Category.Bridge,Category.Bubble Tea Shop,Category.Buffet,Category.Building,Category.Burger Joint,Category.Burrito Place,Category.Bus Line,Category.Bus Station,Category.Bus Stop,Category.Business Service,Category.Butcher,Category.Cafeteria,Category.Café,Category.Cajun / Creole Restaurant,Category.Camera Store,Category.Candy Store,Category.Cantonese Restaurant,Category.Caribbean Restaurant,Category.Carpet Store,Category.Cheese Shop,Category.Chinese Restaurant,Category.Chiropractor,Category.Chocolate Shop,Category.Church,Category.Clothing Store,Category.Cocktail Bar,Category.Coffee Shop,Category.College Arts Building,Category.College Auditorium,Category.College Cafeteria,Category.College Gym,Category.College Rec Center,Category.College Stadium,Category.College Theater,Category.Colombian Restaurant,Category.Comedy Club,Category.Comfort Food Restaurant,Category.Comic Shop,Category.Concert Hall,Category.Construction & Landscaping,Category.Convenience Store,Category.Cosmetics Shop,Category.Costume Shop,Category.Creperie,Category.Cuban Restaurant,Category.Cupcake Shop,Category.Curling Ice,Category.Dance Studio,Category.Daycare,Category.Deli / Bodega,Category.Dentist's Office,Category.Department Store,Category.Dessert Shop,Category.Dim Sum Restaurant,Category.Diner,Category.Discount Store,Category.Distribution Center,Category.Dive Bar,Category.Doctor's Office,Category.Dog Run,Category.Doner Restaurant,Category.Donut Shop,Category.Dumpling Restaurant,Category.Eastern European Restaurant,Category.Electronics Store,Category.Elementary School,Category.Escape Room,Category.Ethiopian Restaurant,Category.Event Space,Category.Falafel Restaurant,Category.Farm,Category.Farmers Market,Category.Fast Food Restaurant,Category.Field,Category.Filipino Restaurant,Category.Fish & Chips Shop,Category.Fish Market,Category.Flea Market,Category.Flower Shop,Category.Food & Drink Shop,Category.Food Court,Category.Food Service,Category.Food Truck,Category.Fountain,Category.French Restaurant,Category.Fried Chicken Joint,Category.Frozen Yogurt Shop,Category.Fruit & Vegetable Store,Category.Furniture / Home Store,Category.Gaming Cafe,Category.Garden,Category.Gas Station,Category.Gastropub,Category.Gay Bar,Category.General Entertainment,Category.General Travel,Category.German Restaurant,Category.Gift Shop,Category.Gluten-free Restaurant,Category.Golf Course,Category.Golf Driving Range,Category.Gourmet Shop,Category.Greek Restaurant,Category.Grocery Store,Category.Gym,Category.Gym / Fitness Center,Category.Gym Pool,Category.Harbor / Marina,Category.Hardware Store,Category.Health & Beauty Service,Category.Health Food Store,Category.Historic Site,Category.History Museum,Category.Hobby Shop,Category.Hockey Arena,Category.Home Service,Category.Hong Kong Restaurant,Category.Hookah Bar,Category.Hospital,Category.Hostel,Category.Hot Dog Joint,Category.Hotel,Category.Hotel Bar,Category.Hotpot Restaurant,Category.IT Services,Category.Ice Cream Shop,Category.Indian Restaurant,Category.Indie Mo

* Remove the "Category." in heading.

In [135]:
toronto_grouped = toronto_grouped.rename(columns={'Category.Neighborhood':'Category.Neighborhood in venues'})

In [137]:
toronto_grouped.columns = [col.split(".")[-1] for col in toronto_grouped.columns]
toronto_grouped.head()

,Neighborhood,Accessories Store,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood in venues,New American Restaurant,Newsagent,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South America

* Generate features of neighborhoods with the most venue types.

In [139]:
# The function generate the top * venues in each neighborhood.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [155]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']


for index, neighborhood in enumerate(toronto_grouped['Neighborhood']):
    neighborhoods_venues_sorted.iloc[index, 1:] = return_most_common_venues(toronto_grouped.iloc[index, :], num_top_venues)
# for ind in np.arange(toronto_grouped.shape[0]):
#     neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Café,Coffee Shop,Hotel,Restaurant,Japanese Restaurant,Gym,Asian Restaurant,Steakhouse,American Restaurant,Salad Place
1,Agincourt,Discount Store,Shopping Mall,Badminton Court,Sushi Restaurant,Chinese Restaurant,Newsagent,Supermarket,Department Store,Grocery Store,Bakery
2,Agincourt North,Pharmacy,Intersection,Yoga Studio,Elementary School,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
3,Albion Gardens,Hockey Arena,Hardware Store,Gym Pool,Sushi Restaurant,Yoga Studio,Electronics Store,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop
4,Alderwood,Gym,Convenience Store,Pub,Performing Arts Venue,Elementary School,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


### Clustering

In [200]:
# set number of cluster
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 9, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [201]:
toronto_table.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

In [202]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = pd.DataFrame(kmeans.labels_)

toronto_merged = toronto_table

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752450,-79.329910,1.0,Food & Drink Shop,Park,Yoga Studio,Elementary School,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
1,M4A,North York,Victoria Village,43.730570,-79.313060,9.0,Grocery Store,German Restaurant,Pharmacy,Park,Yoga Studio,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop
2,M5A,Downtown Toronto,Regent Park,43.660690,-79.360310,0.0,Coffee Shop,Restaurant,Thai Restaurant,Food Truck,Auto Dealership,Indian Restaurant,Beer Store,Pub,Electronics Store,Sushi Restaurant
3,M5A,Downtown Toronto,Harbourfront,43.655120,-79.362640,0.0,Coffee Shop,Breakfast Spot,Yoga Studio,Italian Restaurant,Pub,Restaurant,Distribution Center,Electronics Store,Event Space,Food Truck
4,M6A,North York,Lawrence Manor,43.714925,-79.449739,0.0,Fast Food Restaurant,Restaurant,Coffee Shop,Discount Store,Grocery Store,Mobile Phone Shop,Bookstore,Sandwich Place,Gas Station,Department Store


In [203]:
# remove 2 neighborhoods where no venue found.
toronto_merged.dropna(inplace=True)

## Answer of the Section 3

In [205]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=False)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters